In [4]:
from google.colab import files
uploaded = files.upload()

Saving RedSox_2024_Batting_Stats.xlsx to RedSox_2024_Batting_Stats.xlsx


In [5]:
from google.colab import files
uploaded = files.upload()

Saving RedSox_2024_CombinedStats.csv to RedSox_2024_CombinedStats.csv


In [6]:
# Red Sox 2024 EDA - Google Colab Version

# Install libraries
!pip install pybaseball openpyxl plotly --quiet

# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pybaseball import batting_stats

# Upload your Excel file manually in the left sidebar, then load it:
file_path = "/content/RedSox_2024_Batting_Stats.xlsx"
df_basic = pd.read_excel(file_path)

# Standardize player names if needed (strip extra spaces)
df_basic['Player'] = df_basic['Player'].str.strip()

# Pull all advanced 2024 stats including players with few plate appearances
df_advanced = batting_stats(2024, qual=0)
df_advanced['Name'] = df_advanced['Name'].str.strip()

# Display both sets of names for troubleshooting
print("Your Excel player names:", df_basic['Player'].sort_values().unique().tolist())
print("pybaseball player names:", df_advanced['Name'].sort_values().unique().tolist())

# Try a basic merge using original player names
df_combined = pd.merge(df_basic, df_advanced, how="left", left_on="Player", right_on="Name")
df_combined.drop(columns=["Name"], inplace=True)

# Save combined dataset (optional)
df_combined.to_csv("RedSox_2024_CombinedStats.csv", index=False)

# Check how many matched
print("Matched rows:", df_combined['WAR'].notna().sum())

# Fix column name if needed
if 'HR_x' in df_combined.columns:
    df_combined.rename(columns={'HR_x': 'HR'}, inplace=True)
elif 'HR_y' in df_combined.columns:
    df_combined.rename(columns={'HR_y': 'HR'}, inplace=True)

# Ensure numeric fields are properly typed
for col in ["HR", "WAR", "wRC+", "OBP", "SLG", "BB%", "K%", "OPS", "BB/K", "RC", "wOBA", "BABIP", "FIP"]:
    if col in df_combined.columns:
        df_combined[col] = pd.to_numeric(df_combined[col], errors='coerce')

# Plot WAR vs HR using Plotly with color theme
if "HR" in df_combined.columns and "WAR" in df_combined.columns:
    fig = px.scatter(
        df_combined, x="HR", y="WAR", hover_name="Player",
        title="WAR vs Home Runs (2024 Red Sox)", labels={"HR": "Home Runs", "WAR": "Wins Above Replacement"},
        color="Player", color_discrete_sequence=px.colors.qualitative.Dark24,
        width=800, height=500
    )
    fig.update_layout(showlegend=False)
    fig.show()
    print("\n💡 WAR vs HR Insight: Higher WAR generally means more overall value. 2+ WAR = solid regular. 5+ = All-Star level. HR boosts WAR but isn't the only factor.")

# BB% vs K% using Plotly with color theme
if "BB%" in df_combined.columns and "K%" in df_combined.columns:
    fig = px.scatter(
        df_combined, x="BB%", y="K%", hover_name="Player",
        title="Walk Rate vs Strikeout Rate (2024 Red Sox)", labels={"BB%": "Walk %", "K%": "Strikeout %"},
        color="Player", color_discrete_sequence=px.colors.qualitative.Safe,
        width=800, height=500
    )
    fig.update_layout(showlegend=False)
    fig.show()
    print("\n💡 BB% vs K% Insight: Higher BB% and lower K% shows plate discipline. BB% > 10% is strong. K% < 20% is considered good.")

# Bar plots for various performance metrics with descriptions and vibrant colors
metrics = {
    "OBP": "💡 OBP Insight: .320 is average. .360+ is excellent at getting on base.",
    "SLG": "💡 SLG Insight: .400 is average. .500+ = serious power hitter.",
    "OPS": "💡 OPS Insight: .750 is average. .850+ = very productive hitter.",
    "BB/K": "💡 BB/K Insight: >0.5 is good. 1.0+ shows great control.",
    "RC": "💡 RC Insight: Total runs a player creates. Higher = more valuable offensively.",
    "WAR": "💡 WAR Insight: 2+ = solid starter. 5+ = All-Star. 8+ = MVP tier.",
    "wOBA": "💡 wOBA Insight: .320 is average. .370+ = elite hitter.",
    "BABIP": "💡 BABIP Insight: .300 is average. Higher may mean good contact or luck.",
    "FIP": "💡 FIP Insight: Pitcher-only ERA. 4.00 = average. <3.50 = very good."
}

palette_cycle = px.colors.qualitative.Vivid

for i, (metric, desc) in enumerate(metrics.items()):
    if metric in df_combined.columns:
        df_sorted = df_combined.sort_values(metric, ascending=False).head(10)
        fig = px.bar(
            df_sorted, x=metric, y="Player", orientation='h', text=metric,
            title=f"Top 10 Players by {metric} (2024 Red Sox)",
            labels={metric: metric, "Player": "Player"},
            color="Player",
            color_discrete_sequence=palette_cycle
        )
        fig.update_traces(texttemplate='%{text:.3f}', textposition='outside')
        fig.update_layout(yaxis={'categoryorder': 'total ascending'}, height=500, showlegend=False)
        fig.show()
        print(desc)

# SECOND DASHBOARD (non-interactive): OBP vs SLG with WAR sizing
if "OBP" in df_combined.columns and "SLG" in df_combined.columns:
    plt.figure(figsize=(10,6))
    sns.set_palette("husl")
    sns.scatterplot(data=df_combined, x="OBP", y="SLG", size="WAR", legend=False)
    plt.title("OBP vs SLG with WAR Bubble Size (2024 Red Sox)")
    plt.xlabel("On-Base Percentage (OBP)")
    plt.ylabel("Slugging Percentage (SLG")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Defensive Stats Preview (limited in pybaseball)
def_fields = ["Player", "Def", "UZR", "DRS"]
def_data = df_combined[[col for col in def_fields if col in df_combined.columns]]
display(def_data.sort_values(by="Def", ascending=False).head(10))


Your Excel player names: ['Bobby Dalbec', 'Ceddanne Rafaela', 'Connor Wong', 'Danny Jansen', 'David Hamilton', 'Dominic Smith', 'Enmanuel Valdez', 'Garrett Cooper', 'Jamie Westbrook', 'Jarren Duran', 'Masataka Yoshida', 'Mickey Gasper', 'Nick Sogard', 'Pablo Reyes', 'Rafael Devers', 'Reese McGuire', 'Rob Refsnyder', 'Romy González', 'Trevor Story', 'Triston Casas', 'Tyler Heineman', "Tyler O'Neill", 'Vaughn Grissom', 'Wilyer Abreu', 'Zack Short']
pybaseball player names: ['A.J. Minter', 'A.J. Puk', 'AJ Smith-Shawver', 'Aaron Ashby', 'Aaron Brooks', 'Aaron Bummer', 'Aaron Civale', 'Aaron Hicks', 'Aaron Judge', 'Aaron Nola', 'Aaron Schunk', 'Abner Uribe', 'Abraham Toro', 'Adael Amador', 'Adam Cimber', 'Adam Duvall', 'Adam Frazier', 'Adam Kloffenstein', 'Adam Mazur', 'Adam Oller', 'Adam Ottavino', 'Adbert Alzolay', 'Addison Barger', 'Adley Rutschman', 'Adolis Garcia', 'Adrian Del Castillo', 'Adrian Houser', 'Adrian Morejon', 'Akil Baddoo', 'Alan Busenitz', 'Alan Trejo', 'Albert Suarez', '


💡 WAR vs HR Insight: Higher WAR generally means more overall value. 2+ WAR = solid regular. 5+ = All-Star level. HR boosts WAR but isn't the only factor.



💡 BB% vs K% Insight: Higher BB% and lower K% shows plate discipline. BB% > 10% is strong. K% < 20% is considered good.


💡 OBP Insight: .320 is average. .360+ is excellent at getting on base.


💡 OPS Insight: .750 is average. .850+ = very productive hitter.


💡 BB/K Insight: >0.5 is good. 1.0+ shows great control.


💡 WAR Insight: 2+ = solid starter. 5+ = All-Star. 8+ = MVP tier.


💡 wOBA Insight: .320 is average. .370+ = elite hitter.


💡 BABIP Insight: .300 is average. Higher may mean good contact or luck.


,Player,Def
0,Jarren Duran,7.2
5,Wilyer Abreu,5.5
13,Reese McGuire,3.7
15,Trevor Story,2.6
1,Ceddanne Rafaela,2.3
7,David Hamilton,2.0
18,Nick Sogard,1.6
24,Tyler Heineman,1.5
21,Jamie Westbrook,0.2
17,Danny Jansen,-0.6
